In [8]:
from src.zero_shooter import TaxZeroShot
from src.dataset import WebOfScience
from globals import Paths

train_data = WebOfScience('train', topn=200, embeddings_precomputed=True)

tax_zero_shooter = TaxZeroShot(
    train_data.tax_tree,
    f'{Paths.SAVE_DIR}/label_alphas_WebOfScience.json',
    freeze_zstc=True,
    freeze_usp=True
)

In [13]:
list(p for p in tax_zero_shooter.zstc.encoder.model.parameters() if p.requires_grad)

[]

In [10]:
list(tax_zero_shooter.USP.sigmoid_gate_model.parameters())

[Parameter containing:
 tensor([0.3870, 0.4240, 0.1770, 0.3030, 0.3530, 0.3250, 0.5020, 0.0160, 0.2530,
         0.2610, 0.3190, 0.2480, 0.4440, 0.2700, 0.2910, 0.2680, 0.3410, 0.3330,
         0.0290, 0.2850, 0.0130, 0.0110, 0.5220, 0.4360, 0.2250, 0.0050, 0.3040,
         0.0010, 0.0180, 0.3420, 0.3000, 0.1680, 0.4190, 0.3640, 0.4630, 0.6230,
         0.4430, 0.3620, 0.3750, 0.3330, 0.2460, 0.2730, 0.3170, 0.2980, 0.3310,
         0.3410, 0.3040, 0.4630, 0.2580, 0.2010, 0.3490, 0.3010, 0.4070, 0.3370,
         0.0270, 0.3650, 0.0060, 0.2850, 0.3020, 0.0060, 0.3170, 0.2890, 0.3020,
         0.2610, 0.2120, 0.3570, 0.0090, 0.1520, 0.2420, 0.4590, 0.2410, 0.2460,
         0.4190, 0.2870, 0.3190, 0.2820, 0.0010, 0.4170, 0.2660, 0.0180, 0.5250,
         0.0130, 0.1810, 0.3160, 0.3040, 0.3590, 0.4040, 0.1030, 0.3780, 0.2670,
         0.3570, 0.2770, 0.0310, 0.4140, 0.2850, 0.0030, 0.4230, 0.2430, 0.1900,
         0.3030, 0.2940, 0.2820, 0.2360, 0.2320, 0.1420, 0.1490, 0.0070, 0.2410,
     

In [15]:
params = list(p for p in tax_zero_shooter.zstc.encoder.model.parameters() if p.requires_grad) + \
list(p for p in tax_zero_shooter.USP.sigmoid_gate_model.parameters() if p.requires_grad)
params

[]

In [16]:
import pandas as pd

In [18]:
a = {'a': 1, 'b': 2, 'c': 3}
x = pd.DataFrame([a])
x

,a,b,c
0,1,2,3


In [21]:
b = {'a': 4, 'b': 5, 'c': 6}

x = pd.concat([x, pd.DataFrame([b])], ignore_index=True)
x

,a,b,c
0,1,2,3
1,4,5,6
